In [3]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [23]:
Nx = 5
Ne = 500
X = randn(Nx, Ne).^2
Xvalid = randn(Nx, Ne).^2
λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [24]:
C = SparseRadialMapComponent(Nx, -1)

Sparse radial map component of dimension 5 and order p = [-1, -1, -1, -1, -1]


In [25]:
@time optimize(C, X, 1, 0, "kfolds", λ, δ, γ)

size(error[2]) = (5,)
size(valid_error) = (5, 5)
size(error[2]) = (5,)
size(valid_error) = (5, 5)
size(error[2]) = (5,)
size(valid_error) = (5, 5)
size(error[2]) = (5,)
size(valid_error) = (5, 5)
size(error[2]) = (5,)
size(valid_error) = (5, 5)
mean_valid_error = [0.8740657790966676, 0.8828394243234925, 0.8842764390272961, 0.8877451841421637, 0.8900777637400671]
opt_families = 1


LoadError: [91mAssertionError: Error in x_off[39m

In [30]:
@testset "Test greedyfit for a fixed number of family of features" begin

    for Nx=1:5
        Ne = 100

        X = randn(Nx, Ne) .* randn(Nx, Ne)

        λ = 0.0
        δ = 1e-8
        γ = 2.0

        for pdiag=0:3
            for poff=0:3
                for k=0:Nx-1
                    Cgreedy, ϵgreedy = greedyfit(Nx, pdiag, poff, deepcopy(X), k, λ, δ, γ)
                    C = SparseRadialMapComponent(Nx, Cgreedy.p)
                    center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)
                    x_opt = optimize(C, X, λ, δ)
                    modify_a!(C, x_opt)
                    @test isapprox(ϵgreedy[end], negative_likelihood(C, X), atol = 1e-10)
                    @test norm(Cgreedy.a - C.a)<1e-10
                end
            end
        end

        for poff=0:3
            for pdiag=0:3
                for k=0:Nx-1
                    Cgreedy, ϵgreedy = greedyfit(Nx, pdiag, poff, deepcopy(X), k, λ, δ, γ)
                    C = SparseRadialMapComponent(Nx, Cgreedy.p)
                    center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)
                    x_opt = optimize(C, X, λ, δ)
                    modify_a!(C, x_opt)
                    @test isapprox(ϵgreedy[end], negative_likelihood(C, X), atol = 1e-10)
                    @test norm(Cgreedy.a - C.a)<1e-10
                end
            end
        end
    end
end

Test Summary:                                           | Pass  Total
Test greedyfit for a fixed number of family of features |  960    960


Test.DefaultTestSet("Test greedyfit for a fixed number of family of features", Any[], 960, false)

In [19]:
@time Cgreedy, error = greedyfit(Nx, 2, 2, deepcopy(X), 3, λ, δ, γ)

  0.018648 seconds (91.66 k allocations: 11.540 MiB)


(Sparse radial map component of dimension 5 and order p = [2, -1, 2, 2, 2]
, [0.16149625717495586, 0.15955843305309023, 0.1513458062113992, 0.14608833904374321])

In [20]:
@time Cgreedy, error = greedyfit(Nx, 2, 0, deepcopy(X), deepcopy(Xvalid), 4, λ, δ, γ; verbose = false)

  0.004808 seconds (54.84 k allocations: 5.174 MiB)


(Sparse radial map component of dimension 5 and order p = [2, 2, 2, 2, 0]
, ([0.9522111470540667, 0.9468735067392048, 0.9382460125763467, 0.9339206542025913, 0.9330111942049354], [0.8123465391957869, 0.8138996919068819, 0.8178551591012666, 0.8222529910064743, 0.8217679113283066]))

In [57]:
C = SparseRadialMapComponent(Nx, Cgreedy.p)

5-th component of a Sparse KR rearrangement of order p = [2, 2, -1, 2, 2]


In [58]:
@time center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)

  0.000122 seconds (44 allocations: 62.719 KiB)


In [60]:
@time x_opt = optimize(C, X, λ, δ)
modify_a!(C, x_opt)

  0.002689 seconds (18.33 k allocations: 2.522 MiB)


In [65]:
negative_likelihood(C, X)

0.4260154881199424

In [66]:
ncoeff(C)

LoadError: [91mMethodError: no method matching ncoeff(::SparseRadialMapComponent)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  ncoeff([91m::LinHermiteMapComponent[39m) at /media/mat/HDD/AdaptiveTransportMap/src/hermitemap/linhermitemapcomponent.jl:32[39m
[91m[0m  ncoeff([91m::HermiteMapComponent[39m) at /media/mat/HDD/AdaptiveTransportMap/src/hermitemap/hermitemapcomponent.jl:85[39m
[91m[0m  ncoeff([91m::Int64[39m, [91m::Array{Int64,N} where N[39m) at /media/mat/HDD/AdaptiveTransportMap/src/radialmap/weights.jl:349[39m